In [ ]:
"""
Colab Whisper Transcriber
────────────────────────────────────────
1. 安裝依賴（僅第一次執行需要）
2. 將音檔拖到 Colab 左側 Files 或用 upload 視窗上傳
3. 自動轉成 16 kHz / mono WAV ➜ Whisper (zh-TW) ➜ 簡→繁 ➜ .txt
Author: Hsiu-Chi Tsai
License: MIT
"""

# ── 1. 安裝系統與 Python 套件 ────────────────────
!apt-get -y update && apt-get -y install -y ffmpeg
!pip install --quiet --upgrade openai-whisper pydub opencc-python-reimplemented

# ── 2. 匯入所需函式庫 ────────────────────────────
import os
import tempfile
import torch
from typing import Optional

from google.colab import files
from pydub import AudioSegment
import whisper
from opencc import OpenCC

# ── 3. 工具函式 ──────────────────────────────────
def convert_to_wav(src_path: str, target_sr: int = 16_000) -> str:
    """
    將任何格式音檔轉成 16 kHz・mono・16-bit PCM WAV。
    回傳暫存檔路徑，處理過後自動刪除原檔不會影響。
    """
    audio = AudioSegment.from_file(src_path)
    audio = (
        audio.set_frame_rate(target_sr)
             .set_channels(1)
             .set_sample_width(2)        # 16-bit
    )
    wav_path = tempfile.mktemp(suffix=".wav")
    audio.export(wav_path, format="wav")
    return wav_path


def transcribe_zh_tw(wav_path: str, model_size: str = "large") -> str:
    """
    使用 Whisper 轉錄 + OpenCC 簡→繁。
    model_size 可選 tiny/base/small/large/large。
    """
    print(f"⏳ Loading Whisper {model_size} model …")
    model = whisper.load_model(model_size)
    print("🔊 Transcribing …")
    result = model.transcribe(
        wav_path,
        language="zh",
        fp16=torch.cuda.is_available()
    )
    # Whisper 可能夾雜簡體，後處理轉繁體
    cc = OpenCC("s2t")
    return cc.convert(result["text"])


def save_transcript(text: str, src_filename: str) -> str:
    """
    以來源檔名為基礎，儲存 *_transcript.txt，並回傳檔名。
    """
    stem = os.path.splitext(os.path.basename(src_filename))[0]
    txt_path = f"{stem}_transcript.txt"
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(text)
    return txt_path


# ── 4. 主流程 ────────────────────────────────────
def main(src_path: Optional[str] = None, model_size: str = "large") -> None:
    """
    如果 src_path 為 None，則啟動 files.upload() 讓使用者挑檔；
    否則直接使用指定檔案。
    """
    # 4-1 選取音檔
    if src_path is None:
        uploads = files.upload()
        if not uploads:
            raise RuntimeError("❌ 沒有檔案！請至少上傳一個音檔。")
        src_path = next(iter(uploads))
    if not os.path.exists(src_path):
        raise FileNotFoundError(f"❌ 找不到檔案：{src_path}")

    print(f"\n📥 Source file: {src_path}")

    # 4-2 轉成 WAV
    wav_path = convert_to_wav(src_path)
    print(f"🎧 Converted to WAV: {wav_path}")

    # 4-3 Whisper 轉錄
    transcript = transcribe_zh_tw(wav_path, model_size=model_size)

    # 4-4 儲存・下載
    txt_path = save_transcript(transcript, src_path)
    print("\n==== 轉錄結果（繁體） ====\n")
    print(transcript)
    print(f"\n💾 Saved transcript → {txt_path}")
    files.download(txt_path)


# ── 5. 執行 ─────────────────────────────────────
# 若想手動指定目錄下的檔案，可把 src_path 改成實際路徑；
# 留空代表跳出上傳視窗。
main(src_path=None, model_size="large")
